# EfficientNet B0 Network for the AgNOR classification

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from sklearn.model_selection import train_test_split
import torch.optim as optim
from tqdm.notebook import trange, tqdm
from torchmetrics import Accuracy, F1Score

### Custom Dataset class

We have a total of 27 images (1973 annotaions), which is significantly small amount of data. There is a huge imbalance in the dataset that classes o,1,2 are over-represented. Also, we need to split these data into train, validation & test datasets.

Therefore, we come with oversampling method as a solution for the class imbalance.

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, transform, num_samples=1000):
        self.df = df
        self.num_samples = num_samples
        self.transform = transform
        self.sampled_df = self.init_samples()
        self.df = self.sampled_df


    def init_samples(self):

        classes = self.df.label.unique()
        returnclass = np.random.choice(classes, self.num_samples)
        out = []

        for cl in returnclass:
            out.append(df.query(f"label == {cl}").sample(1))

        sampled_df = pd.concat(out)
        
        return sampled_df

    def __len__(self):
        return len(self.sampled_df)

    def __getitem__(self, idx):
        
        row = self.df.iloc[idx]
        max_x = row['max_x']
        max_y = row['max_y']
        min_x = row['min_x']
        min_y = row['min_y']
        imgname = row['filename']
        label = row['label']

        fullimage = Image.open(imgname).convert('RGB')
        img = fullimage.crop(((min_x, min_y, max_x, max_y)))
        
        img = self.transform(img)
        return img, label

### Augmentation Methods

We also need a solution for limited number of images/annotaions. The best way out is to add more images. Unfortunately, it is out of the equation. Hence, we introduce data augmentation methods, namely Gaussian blur, Colorjitter to overcome this problem.

We have fixed crop size to be fed into our model.

In [ ]:
class Gaussian:
    def __init__(self, kernelsize = (3,3)):
        self.kernel_size = kernelsize

    def __call__(self, image):
        image = np.array(image)
        filteredimg = cv2.GaussianBlur(image, self.kernel_size, 0)
        return Image.fromarray(filteredimg)
    


class Colorjitter:
    def __init__(self, brightness=0, contrast=0, saturation=0):
        self.brightness = brightness
        self.contrast = contrast
        self.saturation = saturation


    def differ_brightness(self, image):
        image = image + 255 * self.brightness
        return np.clip(image, 0, 255).astype(np.uint8)
    

    def differ_contrast(self, image):
        meanvalue = np.mean(image, axis=(0,1), keepdims=True)
        image = (image - meanvalue) * self.contrast + meanvalue
        return np.clip(image, 0, 255).astype(np.uint8)


    def differ_saturation(self, image):
        gray = np.mean(image, axis=2, keepdims=True)
        image = image * (1 + self.saturation) + gray * self.saturation
        return np.clip(image, 0, 255).astype(np.uint8)
    

    def __call__(self, image):
        image = np.array(image)
        image = self.differ_brightness(image)
        image = self.differ_contrast(image)
        image = self.differ_saturation(image)
        return image
    

mytransform = transforms.Compose([
    transforms.Resize((50,50)),
    transforms.Lambda(Gaussian()),
    transforms.Lambda(Colorjitter()),
    transforms.ToTensor()
    ])

### Split Raw Data & make Dataloaders

train 70 % , Validation 15 %, Test 15 %

In [ ]:
my_dataset = pd.read_csv('annotation_frame.csv')


# train 70 % , Validation 15 %, Test 15 %
traindata, testdata = train_test_split(my_dataset, test_size=0.3, random_state=56)
validata, testdata = train_test_split(testdata, test_size=0.5, random_state=56)

train_dataset = MyDataset(traindata, transform=mytransform)
val_dataset = MyDataset(validata, transform=mytransform)
test_dataset = MyDataset(testdata, transform=mytransform)

trainloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valiloader = DataLoader(val_dataset, batch_size=4, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

### Initialising the model

For this task, we have used EfficientNet B0 model as the classifier. Model's backbone is frozen. Output layer is changed to 12 classes to match our task. So that we only need to train weights for the output layer.

In [ ]:
model = efficientnet_b0(weights='IMAGENET1K_V1')

#print(model)


# Freezing layers
for param in model.parameters():
    param.requires_grad = False

classes = 12
model.classifier[-1] = torch.nn.Linear(in_features=1280, out_features=classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

### Loss Function & Optimizer

Cross Entrophy loss is a convex loss function which commonly used in neural networks.

The loss function is combined with Adam optimzer to update weights and biases.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training & Validation Loops

Accuracy score from torchmetrics is used to evaluate models in validation phase. The model with lowest validation loss is saved in the disk.

In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in tqdm(dataloader, desc='Training'):
        inputs, labels = inputs.to(device), labels.to(device)
        # making gradients zero
        optimizer.zero_grad()
        # forward pass
        outputs = model(inputs)
        # calculate loss
        loss = criterion(outputs, labels)
        #back propagation
        loss.backward()
        # updating parameters
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

In [ ]:
def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()
    val_loss = 0.0
    accuracy = Accuracy(task='multiclass', num_classes=12).to(device)
    accuracy.reset()

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc='Validation'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            accuracy.update(outputs, labels)

    epoch_loss = val_loss / len(dataloader.dataset)
    acc = accuracy.compute()
    return epoch_loss, acc

In [ ]:
def n_epochs(model, trainloader, valiloader, optimizer, criterion, device, num_epochs=50):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        
        # Training
        train_loss = train_one_epoch(model, trainloader, optimizer, criterion, device)
        train_losses.append(train_loss)
        print(f"Training Loss: {train_loss}")

        # Validation
        val_loss, val_acc = validate_one_epoch(model, valiloader, criterion, device)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

        # Best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model_cl.pth')

    return train_losses, val_losses, val_accuracies


### Training

In [ ]:
train_losses, val_losses, val_accuracies = n_epochs(model, trainloader, valiloader, optimizer, criterion, device)

### Visualising Tr_Loss & Vl_Loss over epochs

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()

### Testing

F1 score is used to evaluate the final model we saved from training. Here, we can grasp how well the model predicts, generalise and etc.

In [ ]:
best_model = efficientnet_b0(weights=None)
best_model.classifier[-1] = torch.nn.Linear(in_features=1280, out_features=classes)
best_model.load_state_dict(torch.load('best_model_cl.pth'))
best_model.to(device)
best_model.eval()

f1_score = F1Score(num_classes=classes, task="multiclass").to(device)

def test_model(model, testloader, device, metric):
    metric.reset()

    with torch.no_grad():
        for inputs, labels in tqdm(testloader, desc='Testing'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            metric.update(outputs, labels)

    f1 = metric.compute()
    return f1


f1 = test_model(best_model, testloader, device, f1_score )
print(f"F1 score is :{f1}")